In [2]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
spacex_df=pd.read_csv("spacex_csv_file.csv")

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Use folium.Circle to add a highlighted circle area with a text label on a specific coordinate.

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Define the coordinates and launch site names
coordinates = [
    [29.559684888503615, -95.0830971930759, 'NASA JSC'],
    [28.562302, -80.577356, 'CCAFS LC-40'],
    [28.563197, -80.576820, 'CCAFS LC-40'],
    [28.573255, -80.646895, 'KSC LC-39A'],
    [34.632834, -120.610746, 'VAFB SLC-4E']
]

# Initialize the map centered around the first launch site
site_map = folium.Map(location=[29.559684888503615, -95.0830971930759], zoom_start=5)

# Add a Circle and Marker for each launch site
for coord in coordinates:
    lat, lon, site_name = coord
    
    # Add Circle
    circle = folium.Circle(
        location=[lat, lon],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site_name))
    
    # Add Marker
    marker = folium.map.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site_name}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
# Create a MarkerCluster
marker_cluster = MarkerCluster()

# Add a new column 'marker_color' to the dataframe
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add marker_cluster to the current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df dataframe, create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successful or failed
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=(
            f"Launch Site: {record['Launch Site']}<br>"
            f"Class: {record['class']}"
        ),
        icon=folium.Icon(color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

# Display the map
site_map

In [12]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [14]:
# Example coordinates of a launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Example coordinates of the closest coastline point
coastline_lat = 28.56458
coastline_lon = -80.56772

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print("Distance to coastline:", distance_coastline, "km")


Distance to coastline: 0.9022033447773495 km


In [15]:
# Example coordinates of the launch site and the closest coastline point
launch_site_coordinates = [28.563197, -80.576820]
coastline_coordinates = [28.56458, -80.56772]

# Create a folium.PolyLine object using the coordinates
lines = folium.PolyLine(locations=[launch_site_coordinates, coastline_coordinates], weight=1)

# Add the PolyLine to the map
site_map.add_child(lines)